## 데이터 전처리

이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

우선 csv 데이터에서 index와 # of Sunspots 데이터를 추출한 후

index -> time / # of Sunsports -> series

로 변환하여 계산할 예정입니다.

series 값은 min-max normalize를 실시하여 0-1 사이의 값으로 변환하였습니다.

In [ ]:
time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)

  for row in reader:
    ## sunspots과 time_step 추출
    sunspots.append(float(row[2]))
    time_step.append(int(row[0]))

## 데이터 추출
series = np.array(sunspots)
min = np.min(series)
max = np.max(series)
series -= min
series /= max

time = np.array(time_step)

split_time = 3000 # 필요한 값 셋팅
# 데이터 Train / Validation Split
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

학습 데이터 구성을 위해 Sliding Window 기법을 사용하겠습니다.

window 사이즈를 30, batch 사이즈를 32로 구성하여 학습 데이터를 구성하였습니다.


In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)


## 네트워크 정의

tf.keras.models.Sequential을 이용해 네트워크를 정의하겠습니다.

Time Series Forecasting 에서 Sliding Window 기법으로 구성된 데이터의 특성을 고려하여, Conv1D, LSTM, Dense 레이어로 네트워크를 구성했습니다.

<br>

구성된 데이터는 연속적인 특성을 가지고 있습니다. 그렇기에 인접한 데이터 간의 연관성을 활용하여 다음 Sunspot을 예측하는데 사용할 수 있습니다.

Conv1D 레이어는 인접한 데이터간의 위치 관계에서의 특징을 추출하는데 사용합니다.

그래서 네트워크의 가장 첫 부분에만 사용을 하였습니다.

<br>

LSTM 레이어는 RNN 레이어를 응용한 레이어입니다.

LSTM은 RNN 레이어처럼 순차적으로 들어오는 데이터를 처리하기에 적합하며, RNN에 비해서 정보가 장기적으로 전달된다는 특징을 가지고 있습니다.

<br>

데이터 처리와 마지막 출력 값의 형태를 맞추기 위하여 Dense 레이어를 활용하였습니다.

주어진 데이터를 바탕으로 다음 Date의 Sunspot 값을 예측하기 때문에 마지막 Layer의 형태는 1로 고정해줍니다.

그리고 분류가 아니라 예측이기에, 마지막 Dense 레이어의 Activation 함수는 지정하면 안됩니다.

<br>

저는 이렇게 3가지를 활용하여 네트워크를 구성하였으나, LSTM 혹은 Dense 만으로 네트워크를 구성하더라도 유의미한 성능이 나오므로 취향에 따라서 구성하시면 될 것 같습니다.

자세한 내용은 아래에 코드로 남겨두겠습니다.

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

tf.keras.backend.clear_session()
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)


model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=60, kernel_size=10,
                             strides=1, padding="causal",
                             activation="relu", input_shape=[None, 1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    
model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(1)
    ])

model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9), metrics=['mae'])
model.fit(train_set, epochs=50)

## Validation 평가

아까 데이터 구성에서 빼둔 236건의 Validation 데이터가 있습니다.

이 데이터를 활용하여 학습 된 모델의 성능을 평가합니다.

<br>

해당 MAE가 0.12 이하로 떨어진다면 모델 설계 및 학습을 잘 실시한 것 입니다.


In [ ]:
import math

def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast


window_size = 30
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

print(result)


### LR_SCHEDULE 찾기

In [ ]:
# 콜백을 추가하여 학습 속도를 조정해보자.
# epoch number에 따라 lr값이 변경되도록 함.
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

train_set = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer=shuffle_buffer_size)

# RNN - SimpleRNN, Lambda, Dense layers
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  tf.keras.layers.SimpleRNN(40, return_sequences=True),
  tf.keras.layers.SimpleRNN(40),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0)
])

# callback 설정을 통해 learnning rate를 조절하기 위함.
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))

optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])
#그래프 통해서 lre 찾기
# epoch가 어떻게 설정되었는지 플롯해보기
lrs = 1e-8 * (10 ** (np.arange(100) / 20))
plt.semilogx(lrs, history.history["loss"])
plt.axis([1e-8, 1e-3, 0, 300])

# 이렇게 찾은 loss function 중 가장 안정적인 값 선택
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

dataset = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer=shuffle_buffer_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  tf.keras.layers.SimpleRNN(40, return_sequences=True),
  tf.keras.layers.SimpleRNN(40),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0)
])
optimizer = tf.keras.optimizers.SGD(lr=5e-5, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(dataset,epochs=400)

## 검사

In [ ]:
forecast=[]
for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
mae=history.history['mae']
loss=history.history['loss']

epochs=range(len(loss)) # Get number of epochs

#------------------------------------------------
# Plot MAE and Loss
#------------------------------------------------
plt.plot(epochs, mae, 'r')
plt.plot(epochs, loss, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()

epochs_zoom = epochs[200:]
mae_zoom = mae[200:]
loss_zoom = loss[200:]

#------------------------------------------------
# Plot Zoomed MAE and Loss
#------------------------------------------------
plt.plot(epochs_zoom, mae_zoom, 'r')
plt.plot(epochs_zoom, loss_zoom, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()